In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Mato Grosso - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_MT.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value,Mato Grosso - Consumo de Cimento (t)
0,2003-1,50.917240,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.297178,0.745147,6.084564e+07,2.670588e+06,16.831332,5.118946e+07,0.331800,47.470
1,2003-2,50.851586,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.291317,0.745308,6.092768e+07,2.672688e+06,16.835967,5.121614e+07,0.333615,45.387
2,2003-3,50.970549,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.285457,0.745469,6.100973e+07,2.674788e+06,16.840602,5.124282e+07,0.334864,44.907
3,2003-4,51.049978,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.279596,0.745630,6.109177e+07,2.676888e+06,16.845237,5.126950e+07,0.336048,45.467
4,2003-5,51.567529,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.273735,0.745791,6.117381e+07,2.678988e+06,16.849872,5.129619e+07,0.336447,56.246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,119.544326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600607,192.533
236,2022-9,118.223448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.598965,183.895
237,2022-10,117.524152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.596866,180.525
238,2022-11,116.430559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.594246,161.683


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
0,0.001009,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.182887
1,0.000000,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.996341,0.011805,0.005153,0.008692,0.004513,0.005346,0.195974
2,0.001828,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.992682,0.023609,0.010307,0.017385,0.009026,0.010692,0.204977
3,0.003049,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.989023,0.035414,0.015460,0.026077,0.013539,0.016038,0.213516
4,0.011003,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.985364,0.047219,0.020613,0.034770,0.018052,0.021384,0.216392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.692912,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.232479,0.768057,0.988798,0.647567,0.992324,0.990966,0.843701
188,0.693045,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.228761,0.756242,0.987198,0.649169,0.991228,0.989675,0.850718
189,0.701363,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.225042,0.744427,0.985598,0.650771,0.990131,0.988384,0.864656
190,0.702586,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.221324,0.732612,0.983998,0.652373,0.989035,0.987094,0.878935


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      51.972
1      40.729
2      54.446
3      51.788
4      58.702
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Mato Grosso - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
0,0.001009,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.182887
1,0.000000,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.996341,0.011805,0.005153,0.008692,0.004513,0.005346,0.195974
2,0.001828,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.992682,0.023609,0.010307,0.017385,0.009026,0.010692,0.204977
3,0.003049,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.989023,0.035414,0.015460,0.026077,0.013539,0.016038,0.213516
4,0.011003,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.985364,0.047219,0.020613,0.034770,0.018052,0.021384,0.216392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.588452,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,0.621493,0.994318,0.973043,0.688394,0.986108,0.975683,0.938508
158,0.582813,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,0.596535,0.988636,0.974495,0.684427,0.985875,0.976460,0.931775
159,0.576827,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,0.571578,0.982954,0.975947,0.680460,0.985641,0.977238,0.924689
160,0.568447,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,0.546620,0.977272,0.977399,0.676493,0.985408,0.978015,0.918022


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       51.972
1       40.729
2       54.446
3       51.788
4       58.702
        ...   
157     65.293
158     90.338
159     83.188
160    107.230
161    105.397
Name: Mato Grosso - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
126,1.000000,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.366310,0.798772,0.815216,0.971029,0.870123,0.843018,0.868231
127,0.991476,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.388500,0.797810,0.822368,0.966201,0.876689,0.849696,0.879874
128,0.976090,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.410690,0.796849,0.829519,0.961372,0.883254,0.856374,0.890140
129,0.962811,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.432879,0.795888,0.836671,0.956544,0.889819,0.863052,0.900472
130,0.946237,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.455069,0.794926,0.843822,0.951716,0.896384,0.869730,0.912345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.692912,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.232479,0.768057,0.988798,0.647567,0.992324,0.990966,0.843701
188,0.693045,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.228761,0.756242,0.987198,0.649169,0.991228,0.989675,0.850718
189,0.701363,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.225042,0.744427,0.985598,0.650771,0.990131,0.988384,0.864656
190,0.702586,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.221324,0.732612,0.983998,0.652373,0.989035,0.987094,0.878935


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2440345657, 1166112683, 2665377128, 3807456458, 452862105, 2643809941, 2403023524, 1269758199, 2137413046, 3057689567]


Step: 0 ___________________________________________
val_loss: 139.6892547607422
winner_seed: 2440345657


Step: 1 ___________________________________________
val_loss: 93.61954498291016
winner_seed: 1166112683


Step: 2 ___________________________________________
val_loss: 63.98911666870117
winner_seed: 2665377128


Step: 3 ___________________________________________
val_loss: 46.291263580322266
winner_seed: 3807456458


Step: 4 ___________________________________________
val_loss: 72.20362854003906


Step: 5 ___________________________________________
val_loss: 40.514766693115234
winner_seed: 2643809941


Step: 6 ___________________________________________
val_loss: 210.52557373046875


Step: 7 ___________________________________________
val_loss: 117.13175201416016


Step: 8 ___________________________________________
val_loss: 98.07527160644531


Step: 9 ________

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 117ms/step - loss: 6250.6733 - val_loss: 194.9953
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 540.1411 - val_loss: 165.7966
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 629.4790 - val_loss: 232.9019
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 528.3806 - val_loss: 276.3489
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 548.8160 - val_loss: 1251.2335
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 704.6389 - val_loss: 139.8875
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 537.2812 - val_loss: 326.7998
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 554.7641 - val_loss: 471.3119
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 605.8939 - val_loss: 302.5765
Epoch 10/10000
4/4 [==============================] - 0s 9ms/step - l

4/4 [==============================] - 0s 9ms/step - loss: 449.5146 - val_loss: 537.8339
Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 392.7638 - val_loss: 525.6580
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 406.9331 - val_loss: 497.0508
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 408.3913 - val_loss: 837.4653
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 413.2385 - val_loss: 505.5439
Epoch 84/10000
4/4 [==============================] - 0s 8ms/step - loss: 413.9580 - val_loss: 651.3520
Epoch 85/10000
4/4 [==============================] - 0s 8ms/step - loss: 401.2244 - val_loss: 748.9079
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 395.0986 - val_loss: 456.8321
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - loss: 405.6166 - val_loss: 535.7702
Epoch 88/10000
4/4 [==============================] - 0s 9ms/step - loss: 441.0

Epoch 157/10000
4/4 [==============================] - 0s 10ms/step - loss: 556.9945 - val_loss: 261.5544
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 514.9568 - val_loss: 474.8788
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 534.7233 - val_loss: 835.0315
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 579.1017 - val_loss: 199.8910
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 518.3388 - val_loss: 270.3449
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 507.7177 - val_loss: 283.6344
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 496.5589 - val_loss: 329.9911
Epoch 164/10000
4/4 [==============================] - 0s 9ms/step - loss: 603.1423 - val_loss: 443.2881
Epoch 165/10000
4/4 [==============================] - 0s 10ms/step - loss: 539.2653 - val_loss: 143.7482
Epoch 166/10000
4/4 [==========================

Epoch 235/10000
4/4 [==============================] - 0s 10ms/step - loss: 427.4886 - val_loss: 456.0042
Epoch 236/10000
4/4 [==============================] - 0s 10ms/step - loss: 358.2415 - val_loss: 322.6826
Epoch 237/10000
4/4 [==============================] - 0s 9ms/step - loss: 354.3904 - val_loss: 157.7340
Epoch 238/10000
4/4 [==============================] - 0s 10ms/step - loss: 423.0708 - val_loss: 724.9000
Epoch 239/10000
4/4 [==============================] - 0s 10ms/step - loss: 407.3753 - val_loss: 704.6366
Epoch 240/10000
4/4 [==============================] - 0s 10ms/step - loss: 433.3138 - val_loss: 422.4919
Epoch 241/10000
4/4 [==============================] - 0s 9ms/step - loss: 395.6623 - val_loss: 137.4223
Epoch 242/10000
4/4 [==============================] - 0s 11ms/step - loss: 431.1804 - val_loss: 397.1321
Epoch 243/10000
4/4 [==============================] - 0s 10ms/step - loss: 369.3193 - val_loss: 225.5019
Epoch 244/10000
4/4 [===========================

Epoch 313/10000
4/4 [==============================] - 0s 8ms/step - loss: 247.3028 - val_loss: 168.1292
Epoch 314/10000
4/4 [==============================] - 0s 9ms/step - loss: 330.1447 - val_loss: 124.1913
Epoch 315/10000
4/4 [==============================] - 0s 9ms/step - loss: 254.7421 - val_loss: 159.5223
Epoch 316/10000
4/4 [==============================] - 0s 10ms/step - loss: 238.9221 - val_loss: 399.0480
Epoch 317/10000
4/4 [==============================] - 0s 10ms/step - loss: 384.3211 - val_loss: 117.1453
Epoch 318/10000
4/4 [==============================] - 0s 9ms/step - loss: 250.6832 - val_loss: 172.9079
Epoch 319/10000
4/4 [==============================] - 0s 10ms/step - loss: 276.0315 - val_loss: 193.6242
Epoch 320/10000
4/4 [==============================] - 0s 10ms/step - loss: 228.9073 - val_loss: 137.7907
Epoch 321/10000
4/4 [==============================] - 0s 10ms/step - loss: 319.2710 - val_loss: 117.8620
Epoch 322/10000
4/4 [=============================

...

4/4 [==============================] - 0s 9ms/step - loss: 42.2402 - val_loss: 65.7210
Epoch 2024/10000
4/4 [==============================] - 0s 9ms/step - loss: 43.7880 - val_loss: 140.5183
Epoch 2025/10000
4/4 [==============================] - 0s 9ms/step - loss: 42.1911 - val_loss: 135.8300
Epoch 2026/10000
4/4 [==============================] - 0s 9ms/step - loss: 33.3852 - val_loss: 225.4514
Epoch 2027/10000
4/4 [==============================] - 0s 8ms/step - loss: 35.8677 - val_loss: 114.4477
Epoch 2028/10000
4/4 [==============================] - 0s 9ms/step - loss: 52.1591 - val_loss: 96.9058
Epoch 2029/10000
4/4 [==============================] - 0s 8ms/step - loss: 56.2302 - val_loss: 92.7381
Epoch 2030/10000
4/4 [==============================] - 0s 8ms/step - loss: 154.6032 - val_loss: 101.2572
Epoch 2031/10000
4/4 [==============================] - 0s 9ms/step - loss: 112.0825 - val_loss: 113.2492
Epoch 2032/10000
4/4 [==============================] - 0s 9ms/step - los

4/4 [==============================] - 0s 8ms/step - loss: 38.7704 - val_loss: 152.1785
Epoch 2102/10000
4/4 [==============================] - 0s 8ms/step - loss: 37.2978 - val_loss: 152.6158
Epoch 2103/10000
4/4 [==============================] - 0s 8ms/step - loss: 40.6371 - val_loss: 138.8617
Epoch 2104/10000
4/4 [==============================] - 0s 8ms/step - loss: 40.0761 - val_loss: 130.5598
Epoch 2105/10000
4/4 [==============================] - 0s 8ms/step - loss: 33.0697 - val_loss: 129.5912
Epoch 2106/10000
4/4 [==============================] - 0s 8ms/step - loss: 50.7353 - val_loss: 135.9797
Epoch 2107/10000
4/4 [==============================] - 0s 8ms/step - loss: 43.5418 - val_loss: 145.0300
Epoch 2108/10000
4/4 [==============================] - 0s 8ms/step - loss: 50.6827 - val_loss: 146.0225
Epoch 2109/10000
4/4 [==============================] - 0s 8ms/step - loss: 46.0635 - val_loss: 151.2847
Epoch 2110/10000
4/4 [==============================] - 0s 9ms/step - lo

4/4 [==============================] - 0s 9ms/step - loss: 32.8391 - val_loss: 156.3764
Epoch 2180/10000
4/4 [==============================] - 0s 9ms/step - loss: 36.0368 - val_loss: 143.3443
Epoch 2181/10000
4/4 [==============================] - 0s 9ms/step - loss: 27.9375 - val_loss: 144.5412
Epoch 2182/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7195 - val_loss: 140.0479
Epoch 2183/10000
4/4 [==============================] - 0s 9ms/step - loss: 32.3372 - val_loss: 149.1095
Epoch 2184/10000
4/4 [==============================] - 0s 8ms/step - loss: 33.8055 - val_loss: 144.8756
Epoch 2185/10000
4/4 [==============================] - 0s 9ms/step - loss: 28.4825 - val_loss: 136.7690
Epoch 2186/10000
4/4 [==============================] - 0s 8ms/step - loss: 38.0857 - val_loss: 140.3552
Epoch 2187/10000
4/4 [==============================] - 0s 8ms/step - loss: 46.0400 - val_loss: 133.5134
Epoch 2188/10000
4/4 [==============================] - 0s 8ms/step - lo

4/4 [==============================] - 0s 9ms/step - loss: 32.5110 - val_loss: 159.6120
Epoch 2258/10000
4/4 [==============================] - 0s 9ms/step - loss: 32.2772 - val_loss: 159.2431
Epoch 2259/10000
4/4 [==============================] - 0s 9ms/step - loss: 30.6201 - val_loss: 161.4182
Epoch 2260/10000
4/4 [==============================] - 0s 9ms/step - loss: 32.2195 - val_loss: 147.7272
Epoch 2261/10000
4/4 [==============================] - 0s 8ms/step - loss: 30.4766 - val_loss: 147.8067
Epoch 2262/10000
4/4 [==============================] - 0s 9ms/step - loss: 30.5721 - val_loss: 143.5752
Epoch 2263/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.6566 - val_loss: 147.4186
Epoch 2264/10000
4/4 [==============================] - 0s 8ms/step - loss: 26.7651 - val_loss: 148.9843
Epoch 2265/10000
4/4 [==============================] - 0s 9ms/step - loss: 24.1155 - val_loss: 150.0324
Epoch 2266/10000
4/4 [==============================] - 0s 8ms/step - lo

4/4 [==============================] - 0s 8ms/step - loss: 122.8833 - val_loss: 120.1700
Epoch 2335/10000
4/4 [==============================] - 0s 8ms/step - loss: 104.3427 - val_loss: 125.7752
Epoch 2336/10000
4/4 [==============================] - 0s 8ms/step - loss: 109.2796 - val_loss: 121.9924
Epoch 2337/10000
4/4 [==============================] - 0s 8ms/step - loss: 77.3765 - val_loss: 127.6403
Epoch 2338/10000
4/4 [==============================] - 0s 8ms/step - loss: 98.0289 - val_loss: 142.6958
Epoch 2339/10000
4/4 [==============================] - 0s 9ms/step - loss: 97.5145 - val_loss: 125.5848
Epoch 2340/10000
4/4 [==============================] - 0s 8ms/step - loss: 71.7050 - val_loss: 121.3193
Epoch 2341/10000
4/4 [==============================] - 0s 9ms/step - loss: 60.2315 - val_loss: 136.2133
Epoch 2342/10000
4/4 [==============================] - 0s 9ms/step - loss: 62.5313 - val_loss: 119.6920
Epoch 2343/10000
4/4 [==============================] - 0s 9ms/step -

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,101.36541,102.829399,103.374832,103.429741,104.057503,105.019737,105.921097,106.506516,107.07856,107.41758,107.911156,108.478455,109.022606,109.571724,110.100304,110.605408,110.981178,111.243477,111.623329,112.188385,112.819679,113.426399,114.060326,114.884132,115.773293,116.913216,117.803154,118.371223,118.819206,119.057419
Target,120.055,125.769,112.904,123.201,112.725,90.828,96.36,86.444,94.285,98.986,88.072,141.01,122.652,142.145,121.124,130.503,104.115,90.69,102.685,96.144,102.197,106.712,124.057,124.625,133.116,144.31,140.357,152.769,124.038,95.054
Error,18.68959,22.939598,9.529167,19.771255,8.667496,14.191734,9.561096,20.062515,12.793556,8.43158,19.839157,32.53154,13.629395,32.57328,11.023697,19.897598,6.86618,20.553474,8.938332,16.044388,10.622681,6.714401,9.996674,9.740868,17.342705,27.396782,22.553841,34.397774,5.218796,24.003418


In [38]:
display(mae)
display(mape)

16.484087

0.14268945

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1263.3899]] - Target[1290.639]| =  Error: [[27.249146]]; MAPE:[[0.02111291]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[1340.527]] - Target[1367.6490000000001]| =  Error: [[27.12207]]; MAPE:[[0.01983116]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[706.7375]] - Target[789.644]| =  Error: [[82.906494]]; MAPE:[[0.10499225]]


[array([[27.249146]], dtype=float32),
 array([[27.12207]], dtype=float32),
 array([[82.906494]], dtype=float32)]

45.759235

0.048645437